# 01 - Performing a test

## Hardware setup

You must have an electronic audio device that you want to test, like a guitar pedal or some other small signal amplifier. We'll refer to this device as the DUT (device under test).

Next, you'll need a real time audio interface. Something like a Focusrite Scarlett 2i2 will work. (If you don't want to buy a separate audio interface, you may be able to use your computer's builtin audio interface. There may be some challenges associated with that, and you'd have to figure how to configure that on your own. On Linux systems, you might need to learn how to enable [ALSA real-time](https://csound.com/docs/manual/RealTimeLinux.html) or you could try using [JACK Audio Connection Kit](https://jackaudio.org/) instead of ALSA.)

Plug the output of your audio interface into the input of the DUT. Then plug the output of the DUT into the input of the audio interface. This way, you'll be able to generate a signal with your computer, send it through the DUT, and capture the DUT's output onto your computer so you can process it later.

## Getting started with `freqbench`

In [4]:
import freqbench

First, we need to run `freqbench.get_devices()` to see all the audio devices your computer has.

In [7]:
devices = freqbench.get_devices()
devices

DevicesInfo(
  input:
    6: "Loopback: PCM (hw:2,0)"
    7: "Loopback: PCM (hw:2,1)"
    8: "Scarlett 2i2 USB: Audio (hw:3,0)"
    9: "sysdefault"
    14: "default"
  output:
    0: "HDA NVidia: HDMI 0 (hw:1,3)"
    1: "HDA NVidia: HDMI 1 (hw:1,7)"
    2: "HDA NVidia: HDMI 2 (hw:1,8)"
    3: "HDA NVidia: HDMI 3 (hw:1,9)"
    4: "HDA NVidia: HDMI 4 (hw:1,10)"
    5: "HDA NVidia: HDMI 5 (hw:1,11)"
    6: "Loopback: PCM (hw:2,0)"
    7: "Loopback: PCM (hw:2,1)"
    8: "Scarlett 2i2 USB: Audio (hw:3,0)"
    9: "sysdefault"
    10: "front"
    11: "surround40"
    12: "surround51"
    13: "surround71"
    14: "default"
    15: "dmix"
)

You will probably see several different devices under `devices.input` and `devices.output`, and yours will almost certainly be different than mine. I'm going to use "Scarlett 2i2 USB: Audio (hw:3,0)" for my input and output device, so I will use device ID 8, which you can see to the left of the device name above. If you run this notebook, be sure to change the following `input_device` and `output_device` definitions according to your devices.

In [8]:
# Audio devices
input_device = 8
output_device = 8

## Generate a test signal

Now we can generate a test signal to send through the DUT. We can do this with `freqbench.signal.sweep()`.

This function creates a frequency sweep signal, which is basically just a sine wave that starts out at some frequency, `freq0`, and speeds up or slows down over time to end up at some other frequency, `freq1`. In this case, we're sweeping from 0 Hz to 22 kHz to cover slightly more than the audible range of frequencies, which is about 20 Hz to 20 kHz. The signal will span over 10 seconds, indicated by the `time` argument. We also have to define a frame rate for the signal. 44.1 kHz is a typical audio frame rate.

One last detail is that we're scaling the signal by `0.1`. This is because `freqbench.signal.sweep()` generates a signal at the maximum amplitude that digital audio signals can encode. Using the maximum amplitude will most likely create distortion and clipping, so we decrease it.

In [11]:
frame_rate = 44_100
freq0 = 0
freq1 = 22_000
time = 10
test_signal = 0.1 * freqbench.signal.sweep(freq0, freq1, time, frame_rate)

## Running a test

We have a test signal, and now we want to run it through the DUT. But first, it's important to recognize that we'll actually need to run two different tests before generating the frequency response. We already know that we need to capture the output signal from the DUT while we pipe the test signal through it, but we also need to capture the signal that we get when the DUT is taken out of the circuit. This is because the audio interface itself has a frequency response, so we can't just compare the output of the DUT directly with `test_signal`. 

Let's first capture the base signal by removing the DUT from the circuit, and plugging the output of the audio interface directly into the input. Alternatively, if your DUT has a true bypass mode, you can just switch into that mode without unplugging anything. Now let's run a base signal test with `freqbench.run()`. Running the following cell will take 10 seconds.

In [13]:
base_signal = freqbench.run(test_signal, frame_rate, input_device, output_device)

Now let's save the output for later. It's saved as a WAV file, so you can try playing it back to make sure that the test worked. (You might have to unplug your audio interface from your computer momentarily to play it back through your computer's speakers.)

In [15]:
freqbench.save(base_signal, frame_rate, 'test_data/base_signal.wav')

Next, let's plug the DUT back in and run the test on it. Again, this will take 10 seconds.

In [16]:
dut_signal = freqbench.run(test_signal, frame_rate, input_device, output_device)

In [17]:
freqbench.save(dut_signal, frame_rate, 'test_data/dut_signal.wav')

Now we have enough data to generate a frequency response. Go to the next notebook for that: [02 - Generate a frequency response curve](02%20-%20Generate%20a%20frequency%20response%20curve.ipynb)